In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.path import Path
from multiprocess import Pool
import networkx as nx

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *
from learning_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD653/MD653_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD652/MD652_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD642/MD642_anchor.txt
File does not exist: 

In [2]:
n_radial = 4
# radial_bins = np.logspace(0, 2, 10, base=10)
radial_bins = np.linspace(0, 100, n_radial+1)

n_angular = 8
angular_bins = np.linspace(-np.pi, np.pi, n_angular+1)

n_orientation_bins = 10
orientation_bins = np.linspace(-np.pi/2, np.pi/2, n_orientation_bins+1)

size_histogram_bins = np.r_[np.linspace(0, 3000, 10), np.inf]

n_edge_length_bins = 10
edge_length_bins = np.r_[np.linspace(0, 100, n_edge_length_bins), np.inf]

n_edge_direction_bins = 10
edge_direction_bins = np.linspace(-np.pi/2, np.pi/2, n_edge_direction_bins+1)

In [3]:
# def compute_features_regions(stack, sec, region_contours):
#     """
#     Compute features for given regions. Save to disk.

#     Args:
#         region_contours:
#             list of n x 2 arrays.
#     """

#     sys.stderr.write('Processing stack %s, section %d...\n' % (stack, sec))

#     cell_orientations = load_cell_data('orientation', stack, sec)
#     cell_orientations = np.array(map(normalize_angle, cell_orientations))

#     cell_centroids = load_cell_data('centroid', stack, sec)
#     cell_numbers = cell_centroids.shape[0]

#     cells = load_cell_data('cells_aligned_mirrored_padded', stack=stack, sec=sec, ext='bp')
#     cells_flattened = cells.reshape((cells.shape[0], -1))
    
#     cell_sizes = load_cell_data('cellSizes', stack=stack, sec=sec, ext='bp')

#     large_cell_indices = load_cell_data('largeCellIndices', stack=stack, sec=sec, ext='bp')
#     large_cell_centroids = cell_centroids[large_cell_indices]
    
#     small_cell_indices = np.setdiff1d(range(cell_numbers), large_cell_indices)
#     small_cell_centroids = cell_centroids[small_cell_indices]
    
#     large_cell_moments = load_cell_data('largeCellFeatures', stack=stack, sec=sec, ext='bp')
    
#     large_cell_neighbor_indices = load_cell_data('neighborCellIndices', stack=stack, sec=sec, ext='hdf')
#     neighbors = dict(zip(large_cell_indices, large_cell_neighbor_indices))
    
#     large_cell_neighbor_vectors = load_cell_data('neighborVectors', stack=stack, sec=sec, ext='hdf')
#     neighbor_vectors = dict(zip(large_cell_indices, large_cell_neighbor_vectors))
    
#     def compute_features_one_region(cnt):
        
#         region_feature = {}

#         # Get large cells inside the particular contour
#         large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
#         large_cell_indices_inside = large_cell_indices[large_cell_is_inside]
# #         print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

#         # Get small cells inside the particular contour
#         small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
#         small_cell_indices_inside = small_cell_indices[small_cell_is_inside]
# #         print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

#         ############################################
#         ## Compute shape descriptor distribution  ##
#         ############################################

#         large_cell_moments_inside_histograms = \
#         np.array([np.histogram(large_cell_moments[large_cell_is_inside, i], bins=10)[0]
#                   for i in range(7)])
        
#         region_feature['huMomentsHist'] = large_cell_moments_inside_histograms
        
#         ##################################
#         # Compute cell size distribution #
#         ##################################

#         region_feature['largeSizeHist'] = np.histogram(cell_sizes[large_cell_indices_inside], 
#                                                            bins=size_histogram_bins)[0]
#         all_cells_indices_inside = np.r_[large_cell_indices_inside, small_cell_indices_inside]
        
#         region_feature['allSizeHist'] = np.histogram(cell_sizes[all_cells_indices_inside],
#                                                          bins=size_histogram_bins)[0]
#         region_feature['largeSizeWeightedHist'] = np.histogram(cell_sizes[all_cells_indices_inside],
#                                                               bins=size_histogram_bins,
#                                                               weights=cell_sizes[all_cells_indices_inside])[0]
        
#         #########################################
#         # Compute cell orientation distribution #
#         #########################################

#         region_feature['largeOrientationHist'] = np.histogram(cell_orientations[large_cell_indices_inside], 
#                                                                   bins=orientation_bins)[0]

#         ##########################################
#         # Construct large-all graph
#         # Start from large cells, link to all cells
#         ##########################################

#         large_all_graph = nx.Graph()

#         for source_sectionwise_idx in large_cell_indices_inside:
            
#             try:
#                 nbrs = neighbors[source_sectionwise_idx]
#             except:
#                 print source_sectionwise_idx, stack, sec
            
#             if len(nbrs) == 0:
#                 continue

#             neighbor_cells_flattened = cells_flattened[nbrs]
#             jacs = compute_jaccard_x_vs_list_v2(cells_flattened[source_sectionwise_idx], neighbor_cells_flattened)

#             for i_sectionwise_idx, vec, jac in zip(nbrs, neighbor_vectors[source_sectionwise_idx], jacs):
#                 length = np.sqrt(np.sum(vec**2))
#                 direction = np.arctan2(vec[1], vec[0])

#                 # Force into between -np.pi/2 and np.pi/2
#                 if direction > np.pi/2:
#                     direction = direction - np.pi
#                 elif direction < -np.pi/2:
#                     direction = direction + np.pi

#                 size_diff = np.abs(cell_sizes[source_sectionwise_idx] - cell_sizes[i_sectionwise_idx])

#                 orientation_diff = cell_orientations[source_sectionwise_idx] - cell_orientations[i_sectionwise_idx]
#                 if orientation_diff > np.pi/2:
#                     orientation_diff -= np.pi
#                 elif orientation_diff < -np.pi/2:
#                     orientation_diff += np.pi

#                 large_all_graph.add_edge(source_sectionwise_idx, i_sectionwise_idx, weight=1, length=length, direction=direction,
#                            orientation_diff=orientation_diff,
#                           size_diff=size_diff,
#                           jaccard=jac)

#         ###########################
#         # Collect edge attributes # 
#         ###########################

# #         all_edge_length = []
# #         all_edge_direction = []
# #         for u, v in g.edges_iter():
# #         #     print u, v
# #             ed = g.get_edge_data(u, v)
# #             all_edge_length.append(ed['length'])
# #             all_edge_direction.append(ed['direction'])

#         #######################################
#         # Compute neighbor distance histogram #
#         #######################################

#     #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

# #         edge_length_histogram, _ = np.histogram(all_edge_length, bins=edge_length_bins)
# #         neighbor_distance_histogram_all_regions.append(edge_length_histogram)

#         ########################################
#         # Compute neighbor direction histogram #
#         ########################################

# #         edge_direction_histogram, _ = np.histogram(all_edge_direction, bins=edge_direction_bins)
# #         neighbor_direction_histogram_all_regions.append(edge_direction_histogram)
        
#         #################################
#         # Subgraph between large cells. #
#         #################################

#         large_large_subgraph = large_all_graph.subgraph(large_cell_indices_inside)

#         all_large_cell_graph_edge_length = []
#         all_large_cell_graph_edge_direction = []
#         all_large_large_graph_edge_jaccard = []
#         all_large_large_graph_edge_sizeDiff = []
#         all_large_large_graph_edge_orientationDiff = []
        
#         for u, v in large_large_subgraph.edges_iter():
#             edge_data = large_large_subgraph.get_edge_data(u, v)
#             all_large_cell_graph_edge_length.append(edge_data['length'])
#             all_large_cell_graph_edge_direction.append(edge_data['direction'])
#             all_large_large_graph_edge_jaccard.append(edge_data['jaccard'])
#             all_large_large_graph_edge_sizeDiff.append(edge_data['size_diff'])
#             all_large_large_graph_edge_orientationDiff.append(edge_data['orientation_diff'])
        
#         region_feature['largeLargeLinkJacHist'] = np.histogram(all_large_large_graph_edge_jaccard, bins=10)[0]
#         region_feature['largeLargeLinkSizeDiffHist'] = np.histogram(all_large_large_graph_edge_sizeDiff, bins=10)[0]
#         region_feature['largeLargeLinkOrientationDiffHist'] = np.histogram(all_large_large_graph_edge_orientationDiff, bins=10)[0]
#         region_feature['largeLargeLinkLenHist'] = np.histogram(all_large_cell_graph_edge_length, bins=edge_length_bins)[0]
#         region_feature['largeLargeLinkDirHist'] = np.histogram(all_large_cell_graph_edge_direction, bins=edge_direction_bins)[0]

#         ###########################
#         # Large/small cells graph #
#         ###########################

#         all_large_small_cell_graph_edge_length = []
#         all_large_small_cell_graph_edge_direction = []

#         for u, v in large_all_graph.edges_iter():
#             if (u in large_cell_indices_inside and v in small_cell_indices_inside) or \
#             (v in large_cell_indices_inside and u in small_cell_indices_inside):
#                 ed = large_all_graph.get_edge_data(u,v)
#                 all_large_small_cell_graph_edge_length.append(ed['length'])
#                 all_large_small_cell_graph_edge_direction.append(ed['direction'])

#         region_feature['largeSmallLinkLenHist'] = np.histogram(all_large_small_cell_graph_edge_length, bins=edge_length_bins)[0]
#         region_feature['largeSmallLinkDirHist'] = np.histogram(all_large_small_cell_graph_edge_direction, bins=edge_direction_bins)[0]

#         return region_feature
    
# #     region_features = [compute_features_one_region(cnt) for cnt in region_contours]
    
#     pool = Pool(15)
#     region_features = pool.map(compute_features_one_region, region_contours)
#     pool.close()
#     pool.join()
        
#     # Saving as pickle is smaller than saving as hdf.
    
# #     fp = get_cell_classifier_data_filepath(what='region_features', stack=stack, sec=sec, ext='hdf')
# #     save_hdf_v2(region_features, fp)
#     fp = get_cell_classifier_data_filepath(what='region_features', stack=stack, sec=sec, ext='pkl')
#     save_pickle(region_features, fp)
    
#     fp = get_cell_classifier_data_filepath(what='region_contours', stack=stack, sec=sec, ext='bp')
#     create_parent_dir_if_not_exists(fp)
#     bp.pack_ndarray_file(region_contours, fp)

In [4]:
patch_size = 224
half_size = patch_size/2
patch_spacing = 56

In [8]:
# for stack in all_annotated_nissl_stacks:  
for stack in ['MD590']:
    
    grid_spec = (patch_size, patch_spacing, ) + metadata_cache['image_shape'][stack]
    grid_locations = grid_parameters_to_sample_locations(grid_spec)

    first_section, last_section = metadata_cache['section_limits'][stack]
    
#     for sec in range(first_section, last_section+1):
    for sec in range(204, 205):

        if is_invalid(sec=sec, stack=stack):
            continue
        
        # Define regions as all patches inside a mask.
            
        mask_tb = DataManager.load_thumbnail_mask_v2(stack, sec)
        region_patch_indices = locate_patches_v2(stack=stack, grid_spec=grid_spec, mask_tb=mask_tb)
        print 'Use %s regions.' % len(region_patch_indices)
        
        region_patch_locations = grid_locations[region_patch_indices]
        ul = region_patch_locations + (-half_size, -half_size)
        ur = region_patch_locations + (-half_size, half_size)
        lr = region_patch_locations + (half_size, half_size)
        ll = region_patch_locations + (half_size, -half_size)
        
        region_contours = np.array(zip(ul, ur, lr, ll))
        
        t = time.time()
#         compute_features_regions(stack, sec, region_contours[:1000])

        sys.stderr.write('Processing stack %s, section %d...\n' % (stack, sec))

        cell_orientations = load_cell_data('orientation', stack, sec)
        cell_orientations = np.array(map(normalize_angle, cell_orientations))

        cell_centroids = load_cell_data('centroid', stack, sec)
        cell_numbers = cell_centroids.shape[0]

        cells = load_cell_data('cells_aligned_mirrored_padded', stack=stack, sec=sec, ext='bp')
        cells_flattened = cells.reshape((cells.shape[0], -1))

        cell_sizes = load_cell_data('cellSizes', stack=stack, sec=sec, ext='bp')

        large_cell_indices = load_cell_data('largeCellIndices', stack=stack, sec=sec, ext='bp')
        large_cell_centroids = cell_centroids[large_cell_indices]

        small_cell_indices = np.setdiff1d(range(cell_numbers), large_cell_indices)
        small_cell_centroids = cell_centroids[small_cell_indices]

        large_cell_moments = load_cell_data('largeCellFeatures', stack=stack, sec=sec, ext='bp')

        large_cell_neighbor_indices = load_cell_data('neighborCellIndices', stack=stack, sec=sec, ext='hdf')
        neighbors = dict(zip(large_cell_indices, large_cell_neighbor_indices))

        large_cell_neighbor_vectors = load_cell_data('neighborVectors', stack=stack, sec=sec, ext='hdf')
        neighbor_vectors = dict(zip(large_cell_indices, large_cell_neighbor_vectors))

        def compute_features_one_region(cnt):

            region_feature = {}

            # Get large cells inside the particular contour
            large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
            large_cell_indices_inside = large_cell_indices[large_cell_is_inside]
    #         print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

            # Get small cells inside the particular contour
            small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
            small_cell_indices_inside = small_cell_indices[small_cell_is_inside]
    #         print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

            ############################################
            ## Compute shape descriptor distribution  ##
            ############################################

            large_cell_moments_inside_histograms = \
            np.array([np.histogram(large_cell_moments[large_cell_is_inside, i], bins=10)[0]
                      for i in range(7)])

            region_feature['huMomentsHist'] = large_cell_moments_inside_histograms

            ##################################
            # Compute cell size distribution #
            ##################################

            region_feature['largeSizeHist'] = np.histogram(cell_sizes[large_cell_indices_inside], 
                                                               bins=size_histogram_bins)[0]
            all_cells_indices_inside = np.r_[large_cell_indices_inside, small_cell_indices_inside]

            region_feature['allSizeHist'] = np.histogram(cell_sizes[all_cells_indices_inside],
                                                             bins=size_histogram_bins)[0]
            region_feature['largeSizeWeightedHist'] = np.histogram(cell_sizes[all_cells_indices_inside],
                                                                  bins=size_histogram_bins,
                                                                  weights=cell_sizes[all_cells_indices_inside])[0]

            #########################################
            # Compute cell orientation distribution #
            #########################################

            region_feature['largeOrientationHist'] = np.histogram(cell_orientations[large_cell_indices_inside], 
                                                                      bins=orientation_bins)[0]

            ##########################################
            # Construct large-all graph
            # Start from large cells, link to all cells
            ##########################################

            large_all_graph = nx.Graph()

            for source_sectionwise_idx in large_cell_indices_inside:

                try:
                    nbrs = neighbors[source_sectionwise_idx]
                except:
                    print source_sectionwise_idx, stack, sec

                if len(nbrs) == 0:
                    continue

                neighbor_cells_flattened = cells_flattened[nbrs]
                jacs = compute_jaccard_x_vs_list_v2(cells_flattened[source_sectionwise_idx], neighbor_cells_flattened)

                for i_sectionwise_idx, vec, jac in zip(nbrs, neighbor_vectors[source_sectionwise_idx], jacs):
                    length = np.sqrt(np.sum(vec**2))
                    direction = np.arctan2(vec[1], vec[0])

                    # Force into between -np.pi/2 and np.pi/2
                    if direction > np.pi/2:
                        direction = direction - np.pi
                    elif direction < -np.pi/2:
                        direction = direction + np.pi

                    size_diff = np.abs(cell_sizes[source_sectionwise_idx] - cell_sizes[i_sectionwise_idx])

                    orientation_diff = cell_orientations[source_sectionwise_idx] - cell_orientations[i_sectionwise_idx]
                    if orientation_diff > np.pi/2:
                        orientation_diff -= np.pi
                    elif orientation_diff < -np.pi/2:
                        orientation_diff += np.pi

                    large_all_graph.add_edge(source_sectionwise_idx, i_sectionwise_idx, weight=1, length=length, direction=direction,
                               orientation_diff=orientation_diff,
                              size_diff=size_diff,
                              jaccard=jac)

            ###########################
            # Collect edge attributes # 
            ###########################

    #         all_edge_length = []
    #         all_edge_direction = []
    #         for u, v in g.edges_iter():
    #         #     print u, v
    #             ed = g.get_edge_data(u, v)
    #             all_edge_length.append(ed['length'])
    #             all_edge_direction.append(ed['direction'])

            #######################################
            # Compute neighbor distance histogram #
            #######################################

        #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

    #         edge_length_histogram, _ = np.histogram(all_edge_length, bins=edge_length_bins)
    #         neighbor_distance_histogram_all_regions.append(edge_length_histogram)

            ########################################
            # Compute neighbor direction histogram #
            ########################################

    #         edge_direction_histogram, _ = np.histogram(all_edge_direction, bins=edge_direction_bins)
    #         neighbor_direction_histogram_all_regions.append(edge_direction_histogram)

            #################################
            # Subgraph between large cells. #
            #################################

            large_large_subgraph = large_all_graph.subgraph(large_cell_indices_inside)

            all_large_cell_graph_edge_length = []
            all_large_cell_graph_edge_direction = []
            all_large_large_graph_edge_jaccard = []
            all_large_large_graph_edge_sizeDiff = []
            all_large_large_graph_edge_orientationDiff = []

            for u, v in large_large_subgraph.edges_iter():
                edge_data = large_large_subgraph.get_edge_data(u, v)
                all_large_cell_graph_edge_length.append(edge_data['length'])
                all_large_cell_graph_edge_direction.append(edge_data['direction'])
                all_large_large_graph_edge_jaccard.append(edge_data['jaccard'])
                all_large_large_graph_edge_sizeDiff.append(edge_data['size_diff'])
                all_large_large_graph_edge_orientationDiff.append(edge_data['orientation_diff'])

            region_feature['largeLargeLinkJacHist'] = np.histogram(all_large_large_graph_edge_jaccard, bins=10)[0]
            region_feature['largeLargeLinkSizeDiffHist'] = np.histogram(all_large_large_graph_edge_sizeDiff, bins=10)[0]
            region_feature['largeLargeLinkOrientationDiffHist'] = np.histogram(all_large_large_graph_edge_orientationDiff, bins=10)[0]
            region_feature['largeLargeLinkLenHist'] = np.histogram(all_large_cell_graph_edge_length, bins=edge_length_bins)[0]
            region_feature['largeLargeLinkDirHist'] = np.histogram(all_large_cell_graph_edge_direction, bins=edge_direction_bins)[0]

            ###########################
            # Large/small cells graph #
            ###########################

            all_large_small_cell_graph_edge_length = []
            all_large_small_cell_graph_edge_direction = []

            for u, v in large_all_graph.edges_iter():
                if (u in large_cell_indices_inside and v in small_cell_indices_inside) or \
                (v in large_cell_indices_inside and u in small_cell_indices_inside):
                    ed = large_all_graph.get_edge_data(u,v)
                    all_large_small_cell_graph_edge_length.append(ed['length'])
                    all_large_small_cell_graph_edge_direction.append(ed['direction'])

            region_feature['largeSmallLinkLenHist'] = np.histogram(all_large_small_cell_graph_edge_length, bins=edge_length_bins)[0]
            region_feature['largeSmallLinkDirHist'] = np.histogram(all_large_small_cell_graph_edge_direction, bins=edge_direction_bins)[0]

            return region_feature

#         region_features = [compute_features_one_region(cnt) for cnt in region_contours]

        pool = Pool(15)
        region_features = pool.map(compute_features_one_region, region_contours)
        pool.close()
        pool.join()

#         # Compared to saving as pkl, saving as hdf results in both smaller file and 20x faster loading.
#         fp = get_cell_classifier_data_filepath(what='region_features', stack=stack, sec=sec, ext='hdf')
#         create_parent_dir_if_not_exists(fp)
#         save_hdf_v2(region_features, fp)
        
#         fp = get_cell_classifier_data_filepath(what='region_contours', stack=stack, sec=sec, ext='bp')
#         bp.pack_ndarray_file(region_contours, fp)

#         del region_features
        
#         sys.stderr.write('Time: %d seconds\n' % (time.time()-t)) # 100 seconds / 46k regions

Use 48771 regions.


Processing stack MD590, section 204...


In [5]:
t = time.time()
q = load_hdf_v2(get_cell_classifier_data_filepath(what='region_features', stack=stack, sec=sec, ext='hdf'))
print time.time() - t

NameError: name 'stack' is not defined